In [ ]:
%pip install joblib librosa numpy pandas --quiet

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
import joblib
import librosa
from audio_feature_extraction import AudioFeatureExtractor
print('Imports OK')

Imports OK


In [ ]:
class VoiceModelTester:
    """Test voice verification model with new audio samples."""

    def __init__(self, model_path: str = 'models/voice_verification_randomforest.pkl',
                 scaler_path: str = 'models/scaler.pkl',
                 feature_columns_path: str = 'models/feature_columns.txt',
                 remove_silence: bool = False):
        self.model_path = Path(model_path)
        self.scaler_path = Path(scaler_path)
        self.feature_columns_path = Path(feature_columns_path)
        self.remove_silence = remove_silence
        self.model = None
        self.scaler = None
        self.feature_columns = None
        self.extractor = None
        self.load_model()

    def load_model(self):
        if not self.model_path.exists():
            raise FileNotFoundError(f'Model file not found: {self.model_path}')
        if not self.scaler_path.exists():
            raise FileNotFoundError(f'Scaler file not found: {self.scaler_path}')
        if not self.feature_columns_path.exists():
            raise FileNotFoundError(f'Feature columns file not found: {self.feature_columns_path}')

        self.model = joblib.load(self.model_path)
        self.scaler = joblib.load(self.scaler_path)
        with open(self.feature_columns_path, 'r') as f:
            self.feature_columns = [line.strip() for line in f.readlines()]
        self.extractor = AudioFeatureExtractor(remove_silence=self.remove_silence)
        print('✓ Model & scaler loaded')

    def predict_audio(self, audio_path: str) -> dict:
        features = self.extractor.extract_features(audio_path)
        if features is None:
            return {'success': False, 'error': 'Failed to extract features'}
        feature_dict = {col: features.get(col, 0.0) for col in self.feature_columns}
        feature_df = pd.DataFrame([feature_dict])
        X = self.scaler.transform(feature_df.values)
        prediction = self.model.predict(X)[0]
        probabilities = self.model.predict_proba(X)[0]
        classes = self.model.classes_ if hasattr(self.model, 'classes_') else list(range(len(probabilities)))
        results = {'success': True, 'audio_path': audio_path, 'predicted_person': prediction, 'probabilities': {}}
        for i, person in enumerate(classes):
            results['probabilities'][person] = float(probabilities[i])
        return results

    def test_single_file(self, audio_path: str, verbose: bool = True):
        audio_path = Path(audio_path)
        if not audio_path.exists():
            print(f'❌ Audio file not found: {audio_path}')
            return None
        if verbose:
            print(f'TESTING: {audio_path.name}')
        results = self.predict_audio(str(audio_path))
        if not results['success']:
            if verbose:
                print('❌', results.get('error'))
            return results
        if verbose:
            print('Predicted:', results['predicted_person'])
            for p, prob in results['probabilities'].items():
                print(f'  {p}: {prob:.4f}')
        return results

## Demo: test a single audio file
Change the `sample_audio` path to a `.wav` file you want to test. Make sure `models/` contains the trained model files.

In [ ]:
# Example usage (edit these paths as needed):
sample_audio = 'audio_data/Kerie/Approve 1.wav'  # change to your file
tester = VoiceModelTester(model_path='models/voice_verification_randomforest.pkl',
                         scaler_path='models/scaler.pkl',
                         feature_columns_path='models/feature_columns.txt',
                         remove_silence=False)
# Run test (uncomment to run)
# res = tester.test_single_file(sample_audio)
# print(res)
print('Demo cell ready. Update paths and uncomment last lines to run the test.')